In [2]:
import sys
import otter

# try:
#   import otterdd
# except ImportError:
#     %pip install otter-grader
#     import otter

grader = otter.Notebook("HWB1.ipynb")

## 🧠 Bonus: Converting Digital Circuits to Transition Systems

### 🔍 Objective

In this assignment, you will implement a method to convert a simple **digital circuit** into a **transition system**. The goal is to capture the behavior of a circuit under all possible input combinations and internal register states.

---

### 🧩 Problem Setup

You are given a class `Circuit` with the following structure:

```python
class Circuit:
    def __init__(self, X, R, Y, update_registers, compute_outputs):
        ...
```

- `X`: Number of binary input bits
- `R`: Number of binary registers (internal state bits)
- `Y`: Number of binary output bits
- `update_registers`: A function that computes the next register state based on inputs and current registers
- `compute_outputs`: A function that computes outputs based on inputs and current registers

Your task is to implement the method:

```python
def to_transition_system(self) -> TransitionSystem:
    ...
```

This method constructs a transition system representing the circuit’s behavior.

---


### 🔧 Transition System Specification

The transition system should include the following components:

#### ✅ States (`S`)
- A state is a pair `(X, R)`, representing a combination of input values and register values.
- Enumerate all possible Boolean combinations.
- Input values and register values should be tuples of Booleans.

#### ✅ Actions (`Act`)
- Actions correspond to all possible values of the input vector `X`.

#### ✅ Transitions (`T`)
- Each transition `(s, a, s')` represents moving from state `s` to `s'` by applying input `a`.
- `s = (X_old, R_old)`, `a = X_new`, `s' = (X_new, update_registers(X_old, R_old))`

#### ✅ Initial States (`I`)
- All states where the register vector is initialized to all `False` (`0`), and `X` can be any combination.

#### ✅ Atomic Propositions (`AP`)
- APs should include:
  - Each input bit: `x1`, `x2`, ..., `xN`
  - Each register bit: `r1`, `r2`, ..., `rM`
  - Each output bit: `y1`, `y2`, ..., `yK`

#### ✅ Labeling Function (`L`)
- Labels for a state `(X, R)` should include:
  - Inputs `xi` that are `1`
  - Registers `ri` that are `1`
  - Outputs `yi` that are `1`


---


### 🧪 Example

For a circuit with:
- 1 input: `X = 1`
- 1 register: `R = 1`
- 1 output: `Y = 1`

If `update_registers(X, R)` = `X`, and `compute_outputs(X, R)` = `R`, then:

- Initial states: `[(True, False), (False, False)]`
- Action space: `[(True,), (False,)]`
- A transition from `(True, False)` with action `(False,)` goes to `(False, True)`.

---

In [20]:
import networkx as nx
import matplotlib.pyplot as plt
from typing import Set, Dict, Tuple, Union, Optional
from queue import Queue

State = Union[str, Tuple]  # A state can be a string or a tuple (location, environment)
Action = str  # Actions are represented as strings
Transition = Tuple[State, Action, State]  # (source_state, action, target_state)
LabelingMap = Dict[State, Set[str]]  # Maps states to atomic propositions


class TransitionSystem:
    """
    A Transition System (TS) representation.

    Attributes:
        S (Set[State]): The set of all states (strings or tuples).
        Act (Set[Action]): The set of all possible actions.
        Transitions (Set[Transition]): The set of transitions, each represented as (state_origin, action, state_target).
        I (Set[State]): The set of initial states.
        AP (Set[str]): The set of atomic propositions.
        _L (LabelingMap): A dictionary mapping states to their respective atomic propositions.
    """

    def __init__(
        self,
        states: Optional[Set[State]] = None,
        actions: Optional[Set[Action]] = None,
        transitions: Optional[Set[Transition]] = None,
        initial_states: Optional[Set[State]] = None,
        atomic_props: Optional[Set[str]] = None,
        labeling_map: Optional[LabelingMap] = None,
    ) -> None:
        """
        Initializes the Transition System.

        :param states: A set of states (each a string or a tuple). Defaults to an empty set.
        :param actions: A set of actions. Defaults to an empty set.
        :param transitions: A set of transitions, each as (state_origin, action, state_target). Defaults to an empty set.
        :param initial_states: A set of initial states. Defaults to an empty set.
        :param atomic_props: A set of atomic propositions. Defaults to an empty set.
        :param labeling_map: A dictionary mapping states to sets of atomic propositions. Defaults to an empty dictionary.
        """
        self.S: Set[State] = set(states) if states is not None else set()
        self.Act: Set[Action] = set(actions) if actions is not None else set()
        self.Transitions: Set[Transition] = set(transitions) if transitions is not None else set()
        self.I: Set[State] = set(initial_states) if initial_states is not None else set()
        self.AP: Set[str] = set(atomic_props) if atomic_props is not None else set()
        self._L: LabelingMap = dict(labeling_map) if labeling_map is not None else {}

    def add_state(self, *states: State) -> "TransitionSystem":
        """
        Adds one or more states to the transition system.

        :param states: One or more states (strings or tuples) to be added.
        :return: The TransitionSystem instance (for method chaining).
        """
        for state in states:
            self.S.add(state)
        return self

    def add_action(self, *actions: Action) -> "TransitionSystem":
        """
        Adds one or more actions to the transition system.

        :param actions: One or more actions (strings) to be added.
        :return: The TransitionSystem instance (for method chaining).
        """
        self.Act.update(actions)
        return self

    def add_transition(self, *transitions: Transition) -> "TransitionSystem":
        """
        Adds one or more transitions to the transition system.
        Ensures that all involved states and actions exist before adding the transitions.

        Each transition must be provided as a tuple of the form `(state_from, action, state_to)`, where:
        - `state_from` is the source state.
        - `action` is the action performed.
        - `state_to` is the resulting state.

        :param transitions: One or more transitions, each as a tuple `(state_from, action, state_to)`.
        :raises ValueError:
            - If a transition is not a tuple of length 3.
            - If `state_from` or `state_to` does not exist in `self.S`.
            - If `action` is not in `self.Act`.
        :return: The `TransitionSystem` instance (for method chaining).
        """
        for transition in transitions:
            if not isinstance(transition, tuple) or len(transition) != 3:
                raise ValueError(f"Invalid transition: {transition}")
            if transition[0] not in self.S:
                raise ValueError(f"State {transition[0]} is not in the transition system.")
            if transition[2] not in self.S:
                raise ValueError(f"State {transition[2]} is not in the transition system.")
            if transition[1] not in self.Act:
                raise ValueError(f"Action {transition[1]} is not in the transition system.")
        self.Transitions.update(transitions)
        return self
            
    def add_initial_state(self, *states: State) -> "TransitionSystem":
        """
        Adds one or more states to the set of initial states.

        :param states: One or more states to be marked as initial.
        :raises ValueError: If any state does not exist in the system.
        :return: The TransitionSystem instance (for method chaining).
        """
        if any(state not in self.S for state in states):
            raise ValueError(f"Initial state {states[0]} must be in the transition system.")
        self.I.update(states)
        return self

    def add_atomic_proposition(self, *props: str) -> "TransitionSystem":
        """
        Adds one or more atomic propositions to the transition system.

        :param props: One or more atomic propositions (strings) to be added.
        :return: The TransitionSystem instance (for method chaining).
        """
        self.AP.update(props)
        return self

    def add_label(self, state: State, *labels: str) -> "TransitionSystem":
        """
        Adds one or more atomic propositions to a given state.

        :param state: The state to label.
        :param labels: One or more atomic propositions to be assigned to the state.
        :raises ValueError: If the state is not in the system or if any label is not a valid atomic proposition.
        :return: The TransitionSystem instance (for method chaining).
        """
        if state not in self.S:
            raise ValueError(f"Cannot set labels for {state}. State is not in the transition system.")
        if any(label not in self.AP for label in labels):
            invalid_labels = {label for label in labels if label not in self.AP}
            raise ValueError(f"Cannot assign labels {invalid_labels}. They are not in the set of atomic propositions (AP).")
        if state not in self._L:
            self._L[state] = set()
        self._L[state].update(labels)
        return self

    def L(self, state: State) -> Set[str]:
        """
        Retrieves the set of atomic propositions that hold in a given state.

        :param state: The state whose atomic propositions are being retrieved.
        :raises ValueError: If the state is not in the transition system.
        :return: A set of atomic propositions associated with the given state.
        """
        if state not in self.S:
            raise ValueError(f"State {state} is not in the transition system.")
        return self._L.get(state, set())

    def pre(self, S: Union[State, Set[State]], action: Optional[Action] = None) -> Set[State]:
        """
        Computes the set of predecessor states from which a given state or set of states can be reached.

        :param S: A single state (string/tuple) or a collection of states.
        :param action: (Optional) If provided, filters only the transitions that use this action.
        :return: A set of predecessor states.
        """
        actions = {action} if action is not None else self.Act
        return {state_from for state_from, a, state_to in self.Transitions if state_to in S and a in actions}

    def post(self, S: Union[State, Set[State]], action: Optional[Action] = None) -> Set[State]:
        """
        Computes the set of successor states reachable from a given state or a collection of states.

        :param S: A single state or a collection of states.
        :param action: (Optional) Filters transitions by this action.
        :return: A set of successor states.
        """
        actions = {action} if action is not None else self.Act
        states = set()
        for  state_from, a, state_to in self.Transitions:
             if (state_from in S or state_from == S) and a in actions:
                states.add(state_to)
        return states

    def reach(self) -> Set[State]:
        """
        Computes the set of all reachable states from the initial states.

        :return: A set of reachable states.
        """
        # BFS to find reachable states
        visited = set()
        reachable = set(self.I)
        q = Queue()
        for state in self.I:
            q.put(state)
        while not q.empty():
            state = q.get()
            if state in visited:
                continue
            visited.add(state)
            successors = self.post(state)
            reachable.update(successors)
            for succ in successors:
                q.put(succ)
        return reachable

    def is_action_deterministic(self) -> bool:
        """
        Checks whether the transition system is action-deterministic.

        A transition system is action-deterministic if:
        - It has at most one initial state.
        - For each state and action, there is at most one successor state.

        :return: True if the transition system is action-deterministic, False otherwise.
        """
        if len(self.I) > 1:
            return False
        source_trans = set()
        for transition in self.Transitions:
            source, action, target = transition
            if (source, action) in source_trans:
                return False
            source_trans.add((source, action))
        return True
        

    def is_label_deterministic(self) -> bool:
        """
        Checks whether the transition system is label-deterministic.

        A transition system is label-deterministic if:
        - It has at most one initial state.
        - For each state, the number of reachable successor states is equal to the number of unique label sets
          of these successor states.

        :return: True if the transition system is label-deterministic, False otherwise.
        """
        if len(self.I) > 1:
            return False
        for state in self.S:
            successors = self.post(state)
            unique_labels = {frozenset(self.L(succ)) for succ in successors}
            if len(successors) != len(unique_labels):
                return False
        return True
                

    def __repr__(self) -> str:
        """
        Returns a string representation of the Transition System.

        :return: A formatted string representation of the TS.
        """
        return (
            f"TransitionSystem(\n"
            f"  States: {self.S}\n"
            f"  Actions: {self.Act}\n"
            f"  Transitions: {len(self.Transitions)}\n"
            f"  Initial States: {self.I}\n"
            f"  Atomic Propositions: {self.AP}\n"
            f"  Labels: {self._L}\n"
            f")"
        )


    def plot(self, title: str = "Transition System", figsize: Tuple[int, int] = (10, 6)) -> None:
        """
        Plots the Transition System as a directed graph.

        :param title: Title of the plot.
        :param figsize: Figure size for the plot.
        """
        G = nx.DiGraph()

        # Add nodes (states)
        for state in self.S:
            label = f"{state}\n{' '.join(self.L(state))}" if self.L(state) else str(state)
            print(label)
            G.add_node(state, label=label, color="blue" if state in self.I else "yellow")

        # Add edges (transitions)
        for state_from, action, state_to in self.Transitions:
            G.add_edge(state_from, state_to, label=action)

        plt.figure(figsize=figsize)
        pos = nx.spring_layout(G)  # Positioning algorithm for layout

        # Draw nodes
        node_colors = [G.nodes[n]["color"] for n in G.nodes]
        nx.draw(G, pos, with_labels=True, labels=nx.get_node_attributes(G, "label"), node_color=node_colors, edgecolors="black", node_size=2000, font_size=10)

        # Draw edge labels (actions)
        edge_labels = {(u, v): d["label"] for u, v, d in G.edges(data=True)}
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=9)

        plt.title(title)
        plt.show()


In [ ]:
# Add your imports here
import itertools



class Circuit:
    def __init__(self, X, R, Y, update_registers, compute_outputs):
        self.X = X
        self.R = R
        self.Y = Y
        self._update_registers = update_registers
        self._compute_outputs = compute_outputs

    def update_registers(self, X, R):
        """
        Evaluates the circuit given input values and register values.

        :return: new_registers
        """
       
        return self._update_registers(X, R)

    def compute_outputs(self, X, R):
        """
        Computes the output values of the circuit given input values and register values.

        :return:  output_values
        """
        return self._compute_outputs(X, R)
    
    def lable_state(self, state):
        lables = set()
        inputs = state[0]
        registers = state[1]
        outputs = self.compute_outputs(inputs, registers)
        lables.update({f'x{index+1}' for index, input_val in enumerate(inputs) if input_val})
        lables.update({f'r{index+1}' for index, register_val in enumerate(registers) if register_val})
        lables.update({f'y{index+1}' for index, output_val in enumerate(outputs) if output_val})
        return lables
    
    def generate_states(self, X, R):
        return set(itertools.product(
            itertools.product([False, True], repeat=X), 
            itertools.product([False, True], repeat=R)
        ))
    
    def to_transition_system(self):
        """
        Converts the circuit to a transition system.

        :return: transition_system
        """
        inputs = set(itertools.product([True, False], repeat=self.X))
        states = self.generate_states(self.X, self.R)
        init_rgisters = tuple([False]* self.R)
        transitions = set()
        for state in states:
            for action in inputs:
                transitions.add((state, action, (action, self.update_registers(state[0], state[1]))))
        # Create a transition system with the given input values, register values, and output values
        init_states = {(input_, init_rgisters) for input_ in inputs}
        atomic_props = set()
        atomic_props.update([f'x{i+1}'for i in range(self.X)])
        atomic_props.update([f'r{i+1}'for i in range(self.R)])
        atomic_props.update([f'y{i+1}'for i in range(self.Y)])
        labling = {state: self.lable_state(state) for state in states}
        ts = TransitionSystem(states, inputs, transitions, init_states, atomic_props, labling)
        
        return ts


## 🔧 Example: Using `Circuit` and Converting to a `TransitionSystem`

This example demonstrates how to define a simple digital circuit, convert it into a transition system, and inspect its components.

### 🧩 Step 1: Define Update and Output Functions

```python
# Update function: register stores the current input
def update_registers(X, R):
    return (X[0],)

# Output function: output is equal to the register value
def compute_outputs(X, R):
    return (R[0],)
```

---

### 🧩 Step 2: Create the Circuit and Convert

```python
# Create a circuit with 1 input, 1 register, 1 output
circuit = Circuit(X=1, R=1, Y=1, update_registers=update_registers, compute_outputs=compute_outputs)

# Convert the circuit to a transition system
ts = circuit.to_transition_system()
```

---

### 🔍 Step 3: Explore the Transition System

```python
# Print states, actions, initial states, and transitions
print("States:", ts.S)
print("Actions:", ts.Act)
print("Initial States:", ts.I)

print("Some Transitions:")
for t in list(ts.Transitions)[:4]:
    print("  ", t)

# Print labels for one state
print("Labels for first state:", ts.L(next(iter(ts.S))))
```

---

### ✅ Expected Output (Example)

```
States: [((False,), (False,)),
         ((False,), (True,)),
         ((True,), (False,)),
         ((True,), (True,))]
Actions: {(False,), (True,)}
Initial States: {((False,), (False,)),
                 ((True,), (False,))}
Some Transitions:
   (((False,), (False,)), (True,), ((True,), (False,)))
   (((False,), (True,)), (True,), ((True,), (False,)))
   (((True,), (False,)), (False,), ((False,), (True,)))
   (((True,), (True,)), (False,), ((False,), (True,)))
Labels for first state: {'x1', 'r1', 'y1'}
```

This illustrates that:
- Each transition updates the register based on the **previous input**
- Labels correctly reflect inputs (`x1`), registers (`r1`), and outputs (`y1`) that are `True`



In [66]:
# Define the update function: the register stores the current input
def update_registers(X, R):
    return (X[0],)  # one-bit register updated to match input

# Define the output function: output is just the current register value
def compute_outputs(X, R):
    return (R[0],)

# Create the circuit: 1 input, 1 register, 1 output
circuit = Circuit(X=1, R=1, Y=1, update_registers=update_registers, compute_outputs=compute_outputs)

# Convert to transition system
ts = circuit.to_transition_system()

# Print details
print("States:", ts.S)
print("Actions:", ts.Act)
print("Initial States:", ts.I)
print("Some Transitions:")
for t in list(ts.Transitions)[:4]:
    print("  ", t)
print("Labels for first state:", ts.L(next(iter(ts.S))))


States: {((True,), (False,)), ((False,), (False,)), ((True,), (True,)), ((False,), (True,))}
Actions: {(False,), (True,)}
Initial States: {((False,), (False,)), ((True,), (False,))}
Some Transitions:
   (((True,), (False,)), (False,), ((False,), (True,)))
   (((False,), (False,)), (True,), ((True,), (False,)))
   (((False,), (False,)), (False,), ((False,), (False,)))
   (((True,), (False,)), (True,), ((True,), (True,)))
Labels for first state: {'x1'}


In [67]:
grader.check("q1")

q1 results: All test cases passed!

## 🎮 Question 2: 7-Boom Counter Circuit

### 🔍 Objective

In this task, you will implement a digital circuit that models the classic **"7-Boom"** game logic:
- Count upward from 0 to 7
- When reaching 7, say "Boom!", then reset to 0

You will model this behavior in a `Circuit` object and convert it to a transition system.

---

### 🧩 Circuit Specification

- **Inputs (`X`)**: 1 bit
  - `x = 1` means increment the counter
  - `x = 0` means hold the current value

- **Registers (`R`)**: 3 bits
  - These bits represent a counter from `0` to `7` (in binary)

- **Output (`Y`)**: 1 bit
  - `y = 1` **only when** the counter reaches 8 (i.e., after 7 + 1)
  - Once the counter reaches 8, it immediately resets to 0

---

### 🧪 Behavior Examples

| Step | Input `x` | Register (binary) | Output `y` |
|------|-----------|-------------------|------------|
| 0    | -         | `000` (0)         | 0          |
| 1    | 1         | `001` (1)         | 0          |
| 2    | 1         | `010` (2)         | 0          |
| ...  | ...       | ...               | ...        |
| 6    | 1         | `110` (6)         | 0          |
| 7    | 1         | `111` (7)         | **1** Boom!|
| 8    | 1         | `000` (0)         | 0          |
| 9    | 0         | `000` (0)         | 0          |

---

### ✅ Task 1

Implement the following functions:

```python
def update_registers_7boom(X, R) -> Tuple[bool, bool, bool]:
    ...
```

```python
def compute_outputs_7boom(X, R) -> Tuple[bool]:
    ...
```

```python
def seven_boom_circuit() -> Circuit:
    ...
```

Where:
- You define `update_registers(X, R)` to implement the counting and reset logic
- You define `compute_outputs(X, R)` to output `y = 1` only when counter == 7

Then, test your circuit using `.to_transition_system()` or `.plot()`.

---

In [112]:
def update_registers_7boom(X, R):
    """
    Update the 3-bit register based on the input x (either 0 or 1).

    Parameters:
        X (Tuple[bool]): Input tuple of length 1 (True = increment, False = no-op)
        R (Tuple[bool, bool, bool]): 3-bit register representing an integer from 0 to 7

    Returns:
        Tuple[bool, bool, bool]: The updated 3-bit register (reset to 0 if value becomes 8)
    """
    if not X[0]:
        return R
    
    all_options = [option[::-1] for option in list(itertools.product([False, True], repeat=3))]
    
    index = all_options.index(R)
    return all_options[(index + 1) % len(all_options)]
                
    
        

def compute_outputs_7boom(X, R):
    """
    Compute the output of the 7-Boom circuit.

    Parameters:
        X (Tuple[bool]): Input tuple of length 1
        R (Tuple[bool, bool, bool]): 3-bit register

    Returns:
        Tuple[bool]: Output y = 1 iff the register value is 7
    """
    return (all(R), )


def seven_boom_circuit():
    """
    Construct and return a Circuit object representing the 7-Boom counter.

    - Inputs: 1-bit (x)
    - Registers: 3-bit counter (to count 0 through 7)
    - Output: 1-bit y = 1 only when counter reaches 8 (then resets)

    Returns:
        Circuit: A fully configured Circuit object for the 7-Boom logic
    """
    # Create the circuit with 1 input, 3 registers, and 1 output
    circuit = Circuit(X=1, R=3, Y=1, update_registers=update_registers_7boom, compute_outputs=compute_outputs_7boom)
    return circuit

In [113]:
R = (False, False, False)
X = (True, )
R = update_registers_7boom((False,), R)
for i in range(7):
    print(R)
    R = update_registers_7boom(X, R)

    

(False, False, False)
(True, False, False)
(False, True, False)
(True, True, False)
(False, False, True)
(True, False, True)
(False, True, True)


## 🔍 Task 2

### Objective

Now that you've implemented the 7-Boom circuit, it's time to analyze how many **distinct states** are reachable in its transition system.

---

### 🔧 Task

Implement the function:

```python
def count_reachable_states() -> int:
    ...
```

---


In [114]:
def count_reachable_states() -> int:
    """
    Constructs the transition system of the 7-Boom circuit and returns
    the number of reachable states from the initial states.
    """
    circuit = seven_boom_circuit()
    ts = circuit.to_transition_system()
    reachable_states = ts.reach()
    return len(reachable_states)  # Return the number of reachable states
    

In [115]:
grader.check("q2")

q2 results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [116]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)